In [54]:
import os
import tensorflow as tf
from tensorflow.image import resize
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from keras.metrics import  Recall, CategoricalAccuracy
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import entropy

from helpers.help import *
np.random.seed(0)

In [55]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'gaussian_ds')
label_dict={'Mild':1,'Moderate':1,'Proliferate_DR':1,'Severe':1,'No_DR':0}

# remove macOS file
folders = os.listdir(path)
# folders.remove('.DS_Store')

# get all the samples
array = []
for i in folders:
    detailPath = os.path.join(path,i)
    for j in os.listdir(detailPath):
        array.append([os.path.join(detailPath,j),label_dict[i.split('.')[0]]])

# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

3662

In [56]:
X,y=dataset[::,0],dataset[::,1]
y = y.astype(int)

#One hot encode the labels
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 10% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.1*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 11% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.11, random_state=42)

In [57]:
#We use the helper function to convert the data into tensorflow dataset objects. Note that , the repeat flag needs
#to be set only for the train set , which by default is true.
#The buid_dataset is a custom function that returns tensor batches
# Prepares the datasets for train, validation and testing

val_dataset=build_dataset(x_val,y_val,repeat=False,batch=64)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=64)

BATCH_SIZE=32
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_train,y_train,batch=BATCH_SIZE)

# input shape for the model
input_shape=train_dataset.element_spec[0].shape[1:]

print(input_shape)

(224, 224, 3)


In [58]:
# load the best model, trained before
model_full_trained = keras.models.load_model("model/model_baseline.keras")

model_al = keras.models.load_model("model/model_al.keras")

# Test full model

In [59]:
y_pred_full = []
y_true_full = []
y_proba_full = []

class_names = {0: 'No DR', 1: 'DR'}

# Iterate over the test dataset
for x_batch, y_batch in test_dataset:
    # Predict probabilities for each batch
    y_test_proba_full = model_full_trained.predict(x_batch)

    y_proba_full.extend(y_test_proba_full)

    # Convert probabilities to class labels (for softmax output)
    y_pred_full.extend(np.argmax(y_test_proba_full, axis=1))

    # Convert true labels from one-hot encoding to class labels
    y_true_full.extend(np.argmax(y_batch.numpy(), axis=1))

# Convert lists to numpy arrays
y_pred_full = np.array(y_pred_full)
y_true_full = np.array(y_true_full)

# Transform 0 and 1 into class names
y_pred_full_names = [class_names[label] for label in y_pred_full]
y_true_full_names = [class_names[label] for label in y_true_full]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


# Test Active Learning Model

In [ ]:
y_pred_al = []
y_true_al = []
y_proba_al = []

# iterate over the batch
for x_batch, y_batch in test_dataset:
    
    # prediction of the model
    y_test_proba_al = model_al.predict(x_batch)

    y_proba_al.extend(y_test_proba_al)

    # convert into the most likely class
    y_pred_al.extend(np.argmax(y_test_proba_al, axis=1))

    # convert the true list into a value for this x
    y_true_al.extend(np.argmax(y_batch.numpy(), axis=1))

y_pred_al = np.array(y_pred_al)
y_true_al = np.array(y_true_al)

# numerical values to strings
y_pred_al_names = [class_names[label] for label in y_pred_al]
y_true_al_names = [class_names[label] for label in y_true_al]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


# Metrics of full trained model

In [61]:
print(model_full_trained.evaluate(test_dataset, verbose=0,return_dict=True))
conf_matrix = confusion_matrix(y_true_full, y_pred_full)
print("\nConfusion Matrix:\n", conf_matrix)

# Print classification report
print("\nClassification Report:\n", classification_report(y_true_full_names, y_pred_full_names))

{'categorical_accuracy': 0.9535518884658813, 'loss': 0.13201043009757996}

Confusion Matrix:
 [[188   7]
 [ 10 161]]

Classification Report:
               precision    recall  f1-score   support

          DR       0.96      0.94      0.95       171
       No DR       0.95      0.96      0.96       195

    accuracy                           0.95       366
   macro avg       0.95      0.95      0.95       366
weighted avg       0.95      0.95      0.95       366



# AUC Full model

In [ ]:
# do one-hot enconding
y_true_one_hot = tf.keras.utils.to_categorical(y_true_full, num_classes=2)


y_proba = np.array(y_proba_full)  

print(y_true_one_hot.shape ,y_proba.shape)

# ROC-AUC score for each class
auc = roc_auc_score(y_true_one_hot, y_proba, average='macro', multi_class='ovr')

print(f"Macro-Averaged ROC-AUC Score: {auc:.4f}")

(366, 2) (366, 2)
Macro-Averaged ROC-AUC Score: 0.9897


# Metrics of AL model

In [63]:
print(model_al.evaluate(test_dataset, verbose=0,return_dict=True))

conf_matrix = confusion_matrix(y_true_al, y_pred_al)
print("\nConfusion Matrix:\n", conf_matrix)

# Print classification report
print("\nClassification Report:\n", classification_report(y_true_al_names, y_pred_al_names, target_names=list(class_names.values())))

{'categorical_accuracy': 0.9535518884658813, 'loss': 0.21879823505878448}

Confusion Matrix:
 [[188   7]
 [ 10 161]]

Classification Report:
               precision    recall  f1-score   support

       No DR       0.96      0.94      0.95       171
          DR       0.95      0.96      0.96       195

    accuracy                           0.95       366
   macro avg       0.95      0.95      0.95       366
weighted avg       0.95      0.95      0.95       366



# AUC AL model

In [ ]:
# do one-hot enconding
y_true_one_hot = tf.keras.utils.to_categorical(y_true_al, num_classes=2)


y_proba_al = np.array(y_proba_al)  

print(y_true_one_hot.shape ,y_proba_al.shape)


# ROC-AUC score for each class
auc = roc_auc_score(y_true_one_hot, y_proba_al, average='macro', multi_class='ovr')

print(f"Macro-Averaged ROC-AUC Score: {auc:.4f}")

(366, 2) (366, 2)
Macro-Averaged ROC-AUC Score: 0.9786
